## Migrador instalaciones régimen especial

In [208]:
import pandas as pd
from erppeek import Client
from datetime import datetime
from tqdm import tqdm

# conexión
try:
    # Due: https://www.python.org/dev/peps/pep-0476
    import ssl
    try:
        _create_unverified_https_context = ssl._create_unverified_context
    except AttributeError:
        # Legacy Python that doesn't verify HTTPS certificates by default
        pass
    else:
        # Handle target environment that doesn't support HTTPS verification
        ssl._create_default_https_context = _create_unverified_https_context
except ImportError:
    pass

c = Client('https://guadiela.gisce.net:8069', 'dbname', 'user')

# migrator
ret = migrator(c, 
             '/home/puig/Fitxers de Medidas/migracion/RE/UPRSL_0304_20191201.24',
             '/home/puig/Fitxers de Medidas/migracion/RE/UFIUPR_0304_20191201.24',
             '/home/puig/Fitxers de Medidas/migracion/RE/CILDATOS_0304_20200110.0',
             'Girona') # municipi
ret

SyntaxError: 'return' outside function (<ipython-input-208-640d3f4fb9a7>, line 20)

In [204]:
def migrator(c, uprsl_name=None, ufiupr_name=None, cildatos_name=None, municipi=None):
    # objects
    partner_obj = c.model('res.partner')
    partner_addr = c.model('res.partner.address')
    try:
        instalacion_re_obj = c.model('giscedata.re')
        unidad_programacion_obj = c.model('giscedata.re.uprs')
    except:
        print('Modul RE no trobat')
        return False
    cups_obj = c.model('giscedata.cups.ps')
    municipi_obj = c.model('res.municipi')
    provincia_obj = c.model('res.country.state')
    polissa_obj = c.model('giscedata.polissa')
    meter_obj = c.model('giscedata.lectures.comptador')
    tarif_obj = c.model('giscedata.polissa.tarifa')
    pricelist_obj = c.model('product.pricelist')
    df_cps = pd.read_csv('cp_espana.csv', sep=';')

    # Preparem datasets
    # df unidades de programación
    df_unidades_de_programacion = pd.read_csv(uprsl_name, 
                      sep = ';',
                     names = ['codigo', 
                            'CIF', 
                            'representante', 
                            'rel_upr_2', 
                            'name_upr', 
                            'desc_upr', 
                            'cod_participante', 
                            'tarifa_o_mercado',
                            'tecnologia', 
                            'exenta_pago', 
                            'data_alta',
                            'data_baixa',
                            '_'], 
                            dtype={'data_alta': str, 'data_baixa': str})
    
    df_unidades_de_programacion.data_alta = df_unidades_de_programacion.data_alta.apply(
        lambda x: '{0}-{1}-{2} 00:00:00'.format(x[:4], x[4:6], x[6:8]))
    df_unidades_de_programacion['data_baixa'] = df_unidades_de_programacion.data_baixa.apply(
        lambda x: '{0}-{1}-{2} 00:00:00'.format(x[:4], x[4:6], x[6:8]))
    df_unidades_de_programacion['cod_participante'] = df_unidades_de_programacion.cod_participante.apply(
        lambda x: str(x).zfill(4))

    # df instalaciones re
    df_instalaciones_re = pd.read_csv(ufiupr_name, 
                     sep = ';', 
                     names = ['codigo_unidad_fisica', 
                            'name_upr', 
                            'name',
                            'descripcio', 
                            'raipre', 
                            'cups', 
                            'cups2', 
                            'data_inici', 
                            'data_final', 
                            '_'])

    if cildatos_name:
        # df cildatos (tipo_punto_frontera)
        df_cildatos = pd.read_csv(cildatos_name, 
                         sep = ';',
                         names = ['cups', 
                                'minetad', 
                                'num_reg_provisional', 
                                'num_reg_definitiva', 
                                'cod_postal', 
                                'potencia_nominal', 
                                'nombre_instalacion', 
                                'psse',
                                'grupo_rd', 
                                'tipo_punto_frontera', 
                                'fecha_alta',
                                'fecha_baja',
                                'tensión_conexión',
                                'fecha_acta_alta',
                                'propiedad_equipo_medida',
                                '_'],
                                 dtype={'tipo_punto_frontera': str, 'potencia_nominal': float})
        df_cildatos['cups'] = df_cildatos['cups'].apply(lambda x: x[:-3])
    
    # merge dfs
    df_instalaciones_re = pd.merge(df_unidades_de_programacion, df_instalaciones_re, on='name_upr', how='right')
    if cildatos_name:
        df_instalaciones_re = pd.merge(df_cildatos, df_instalaciones_re, on='cups', how='left')
    else:
        df_instalaciones_re['tipo_punto_frontera'] = 1
    df_instalaciones_re = df_instalaciones_re[['CIF', 
                                               'representante', 
                                               'name_upr', 
                                               'cod_participante', 
                                               'cups', 
                                               'data_alta', 
                                               'data_baixa', 
                                               'name',
                                               'tipo_punto_frontera',
                                               'potencia_nominal',
                                               'nombre_instalacion',
                                               'grupo_rd',
                                               'cod_postal'
                                              ]]

    # TODO: DEFAULT instalaciones re
    df_instalaciones_re['municipi_name'] = municipi

    # migrar datos
    for instalacion_re_data in tqdm(df_instalaciones_re.T.to_dict().values()):

        # buscar titular para la instalacion re
        titular_name = instalacion_re_data['nombre_instalacion']
        titular_ids = partner_obj.search([('name', '=', titular_name)])
        if not titular_ids:
            # crear titular
            titular_id = partner_obj.create({'name': titular_name, 'vat': ''})
            titular_id = titular_id.id
            p_address_id = partner_addr.create({'name': titular_name, 'partner_id': titular_id})
            print('Titular creat: {}'.format(titular_name))
        else:
            titular_id = titular_ids[0]
        cups_name = instalacion_re_data['cups']
        municipi_name = instalacion_re_data['municipi_name']
        cp = instalacion_re_data['cod_postal']
        cups_id = create_cups(c, cups_name, municipi_name, cp)

        # buscar el contrato del cups
        contrato_ids = polissa_obj.search([('cups', '=', cups_name)])
        if not contrato_ids:
            # crear contrato
            if instalacion_re_data['tipo_punto_frontera'] < 3:
                tariff_name = 'RE12'
            else:
                tariff_name = 'RE'
            tariff_id = tarif_obj.search([('name', '=', tariff_name)])
            pricelist_id = pricelist_obj.search([('name', '=like', 'TARIFAS GENERACI%')])
            assert tariff_id, 'Tarifes de RE no creades!'
            assert pricelist_id, 'Llistes de preu TARIFES GENERACION no creades!'
            tariff_id = tariff_id[0]
            pricelist_id = pricelist_id[0]
            p_id = partner_obj.search([('ref', '=', instalacion_re_data['cod_participante'])])
            if p_id:
                p_id = p_id[0]
            else:
                p_id = False
            if isinstance(instalacion_re_data['data_alta'], float):
                instalacion_re_data['data_alta'] = False
            contrato_vals = {
                    'tarifa': tariff_id, 
                    'cups': cups_id, 
                    'titular': titular_id,
                    'data_alta': instalacion_re_data['data_alta'],
                    'comercialitzadora': p_id,
                    'llista_preu': pricelist_id,
                }
            contrato_id = polissa_obj.create(contrato_vals)
        else:
            contrato_id = contrato_ids[0]

        if "b.1." not in instalacion_re_data['grupo_rd']:
            # Hidroelectriques i demés
            continue

        # crear instalacion_re
        #provincia_ = cups_obj.browse(cups_id).id_provincia.ree_code
        provincia_name = list(set(df_cps[df_cps['codigopostalid'] == int(cp)]['provincia']))
        prov_id = provincia_obj.search([('name', '=', provincia_name[0])])[0]
        provincia_ = provincia_obj.read(prov_id, ['ree_code'])['ree_code']
        
        inst_vals = {
                'name': instalacion_re_data['nombre_instalacion'],
                'cups': cups_id,
                'titular': titular_id,
                'provincia': provincia_,
                'tipo': instalacion_re_data['tipo_punto_frontera'],
                'potencia_nominal': instalacion_re_data['potencia_nominal'],
            }
        instalacion_re_id = instalacion_re_obj.create(inst_vals)

        # UPRS
        representantes_ids = partner_obj.search([('name', '=', instalacion_re_data['representante'])])
        if not representantes_ids:
            representantes_ids = partner_obj.create(
                {
                    'name': instalacion_re_data['representante'], 
                    'ref': instalacion_re_data['cod_participante'],
                    'vat': 'ES' + instalacion_re_data['CIF'],
                }
            )
            representantes_ids = representantes_ids.id
            p_address_id = partner_addr.create({'name': instalacion_re_data['representante'], 'partner_id': representantes_ids})
            print('Representante creado. Validar datos: {}'.format(instalacion_re_data['representante']))
        else:
            representantes_ids = representantes_ids[0]
            ref = partner_obj.read(representantes_ids, ['ref'])['ref']
            if not ref:
                partner_obj.write(representantes_ids, {'ref': instalacion_re_data['cod_participante']})
        unidad_programacion_obj.create(
            {
                'name': instalacion_re_data['name_upr'], 
                'representante': representantes_ids, 
                'inst_re': instalacion_re_id.id,
                'data_alta': instalacion_re_data['data_alta'],
                'data_baixa': instalacion_re_data['data_baixa'],
            }
        )
    
    return df_instalaciones_re

In [203]:
def create_cups(c, cups_name, municipi_name, cp):
    cups_obj = c.GiscedataCupsPs
    municipi_obj = c.ResMunicipi
    cups_ids = cups_obj.search([('name', '=', cups_name)])
    if cups_ids:
        # CUPS existeix
        return cups_ids[0]
    else:
        # CUPS existeix de consum
        cups_ids = cups_obj.search([('name', '=', cups_name[:20] + '0F')])
        if cups_ids:
            cups_data = cups_obj.read(cups_ids[0], ['active', 'carrer', 'cne_anual_activa', 'criteri_regulatori', 'direccio', 'distribuidora_id', 'dp', 'et', 'escala', 'es', 'id_escomesa', 'id_municipi', 'id_poblacio', 'id_provincia', 'numero', 'nv', 'ordre', 'pnp', 'zona', 'tv'])
            for key in ('id_escomesa', 'id_municipi', 'id_poblacio', 'id_provincia', 'tv', 'distribuidora_id'):
                try:
                    x_data = cups_data.pop(key)
                    cups_data.update({key: x_data[0]})
                except:
                    continue
            # Crear CUPS de generacio
            cups_data.pop('id')
            cups_data.update({'name': cups_name})
            cups_id = cups_obj.create(cups_data)
            return cups_id.id
        else:
            # Crear CUPS de 0
            municipi_ids = municipi_obj.search([('name', '=', municipi_name)])
            cups_id = cups_obj.create({'name': cups_name, 'id_municipi': municipi_ids[0], 'dp': cp})
            return cups_id.id

### Subgrups RE
https://energia.gob.es/electricidad/energias-renovables/Paginas/clasificacion-instalaciones-RD413-2014.aspx

- b.1.x: Fotovoltáicas
- b.2.x: Eólica
- b.3.x: Hidrotérmiques, geotérmiques o aerotérmiques
- b.4.x: Hidroeléctriques
- b.5.x: Hidroeléctriques > 10MW

In [184]:
# Cleaner
uprs_obj = c.GiscedataReUprs
re_obj = c.GiscedataRe
pol_obj = c.GiscedataPolissa
cups_obj =c.GiscedataCupsPs

u_ids = uprs_obj.search()
uprs_obj.unlink(u_ids)
re_ids = re_obj.search()
re_obj.unlink(re_ids)
pol_ids = pol_obj.search([('tarifa', '=', 17)])
if pol_ids:
    pol_obj.unlink(pol_ids)
cups_ids = cups_obj.search([('name', '=like', '%1F')])
if cups_ids:
    cups_obj.unlink(cups_ids)
